In [1]:
import pandas as pd

In [32]:
# Points of interest
AreaPOI = pd.read_csv("AreaPOI.csv", usecols=["Name_lang", "Description_lang", "Pos[0]", "Pos[1]", "ContinentID", "AreaID"])

# Continents (and instances?)
# Map.MapDescription0_lang is for the Horde, and MapDescription1_lang is for the Alliance
# Map.ExpansionID is expansion ID (e.g. 7 is BfA)
Map = pd.read_csv("Map.csv", usecols=["ID", "MapName_lang"])

# Zones (and other locations)
AreaTable = pd.read_csv("AreaTable.csv", usecols=["ID", "AreaName_lang"])

In [101]:
AreaPOI_joined = (pd.merge(AreaPOI, Map, how="left", left_on="ContinentID", right_on="ID", suffixes=["", "_Map"])
                  .drop(labels=["ID"], axis=1))
AreaPOI_joined = (pd.merge(AreaPOI_joined, AreaTable, how="left", left_on="AreaID", right_on="ID", suffixes=["", "_AreaTable"])
                  .drop(labels=["ID", "AreaID"], axis=1))
AreaPOI_joined

,Name_lang,Description_lang,Pos[0],Pos[1],ContinentID,MapName_lang,AreaName_lang
0,Anvilmar,NaN,-6134.2800,383.763,0,Eastern Kingdoms,Coldridge Valley
1,Brill,NaN,2249.8500,278.414,0,Eastern Kingdoms,Brill
2,Darkshire,NaN,-10564.7000,-1170.650,0,Eastern Kingdoms,Duskwood
3,Deathknell,NaN,1879.8300,1588.200,0,Eastern Kingdoms,Deathknell
4,Dun Algaz,NaN,-4191.0200,-2373.140,0,Eastern Kingdoms,Dun Algaz
...,...,...,...,...,...,...,...
2669,The Grand Design,NaN,-3825.1338,-2884.962,2481,Sepulcher of the First Ones,NaN
2670,Domination's Grasp,NaN,-5448.2603,-3876.304,2481,Sepulcher of the First Ones,NaN
2671,Heart of Eternity,NaN,-6180.3600,-4098.960,2481,Sepulcher of the First Ones,NaN
2672,The Grand Design,NaN,-6215.8228,-4120.363,2481,Sepulcher of the First Ones,NaN


In [104]:
# Portal entrances and exits
WaypointNode = pd.read_csv("WaypointNode.csv", usecols=["Name_lang", "Field_8_2_0_30080_005", "SafeLocID"]).rename(columns={"Field_8_2_0_30080_005": "NodeType"})
# Portal coordinates
WaypointSafeLocs = pd.read_csv("WaypointSafeLocs.csv", usecols=["ID", "Pos[0]", "Pos[1]", "MapID"])

## Commence processing
WaypointSafeLocs_joined = (pd.merge(WaypointSafeLocs, Map, how="left", left_on="MapID", right_on="ID", suffixes=["", "_Map"])
                               .drop(labels=["ID_Map"], axis=1))
WaypointNode = WaypointNode[WaypointNode.NodeType == 2] # Keep only portal exits
WaypointNode_joined = (pd.merge(WaypointNode, WaypointSafeLocs_joined, how="left", left_on="SafeLocID", right_on="ID")
                       .drop(labels=["SafeLocID", "ID", "NodeType"], axis=1)
                       .rename(columns={"MapID": "ContinentID"}))
WaypointNode_joined

,Name_lang,Pos[0],Pos[1],ContinentID,MapName_lang
0,"Wizard's Sanctum, Portal to Jade Forest",-307.16,-1772.890,870,Pandaria
1,Uldum,-9443.31,-958.360,1,Kalimdor
2,Mount Hyjal,5534.08,-3624.690,1,Kalimdor
3,"Stormwind, The Eastern Earthshrine",-8209.33,428.635,0,Eastern Kingdoms
4,Twilight Highlands,-4890.07,-6626.740,0,Eastern Kingdoms
...,...,...,...,...,...
104,Take the Animaflow Teleporter in Ve'nari's Ref...,3473.16,5955.430,2222,The Shadowlands
105,Take the Waystone in Keeper's Respite to Oribos,-1716.75,1402.680,2222,The Shadowlands
106,Take the portal to Korthia within the Ring of ...,3267.27,5760.560,2222,The Shadowlands
107,Take the portal to Zereth Mortis within the Ri...,-4357.68,800.400,2374,Zereth Mortis


In [106]:
pd.concat([AreaPOI_joined, WaypointNode_joined]).to_csv("JackJackLocations.csv", index=False)